In [ ]:

from pathlib import Path
import sys

# Locate repo root by ascending until we find src/motion_planning
cwd = Path.cwd().resolve()
REPO_ROOT = cwd
for parent in [cwd] + list(cwd.parents):
    if (parent / "src" / "motion_planning").exists():
        REPO_ROOT = parent
        break

# Ensure project src is on sys.path
src_path = REPO_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from motion_planning.online_algorithms.load_problem_to_solve import (
    load_candidates_problem,
    load_graph_problem,
)
from motion_planning.online_algorithms.orb_czl import run_czl_orb
from motion_planning.online_algorithms.orb_bat import run_bat_orb
from motion_planning.online_algorithms.orb_itm import run_itm_online
from motion_planning.online_algorithms.thresholds import czl_thresholds, bat_threshold

CAND_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/problem details"
EDGE_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/graphs with risks"
TIMELINE_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/problem details"
print("Repo root:", REPO_ROOT)
print("sys.path[0]:", sys.path[0])


In [ ]:

# List available candidates/edge files (fallback to rglob if empty)
cand_files = sorted(CAND_PATH.glob("*_candidates.csv"))
edge_files = sorted(EDGE_PATH.glob("*_edge_values.csv"))
timeline_files = sorted(TIMELINE_PATH.glob("*_decision_timeline.csv"))

if not cand_files:
    cand_files = sorted((REPO_ROOT / "src").rglob("*_candidates.csv"))
if not edge_files:
    edge_files = sorted((REPO_ROOT / "src").rglob("*_edge_values.csv"))
if not timeline_files:
    timeline_files = sorted((REPO_ROOT / "src").rglob("*_decision_timeline.csv"))

print("Found candidates:", cand_files[-2:])
print("Found edge files:", edge_files[-2:])
print("Found timelines:", timeline_files[-2:])

sample_cand = cand_files[-1] if cand_files else None
sample_edge = edge_files[-1] if edge_files else None
sample_timeline = timeline_files[-1] if timeline_files else None
print("Using sample cand:", sample_cand)
print("Using sample edge:", sample_edge)
print("Using sample timeline:", sample_timeline)


In [ ]:

if sample_cand:
    cand_problem = load_candidates_problem(sample_cand)
    print("Candidates graph_id:", cand_problem.graph_id)
    print("Capacity:", cand_problem.capacity)
    print("Num epochs:", len(cand_problem.groups))
    print("First epoch candidates:", cand_problem.groups[0].candidates[:3])
else:
    print("No candidates file found. Update paths or generate offline problems.")


In [ ]:

if sample_cand:
    rho_lo, rho_hi, rho_mid = czl_thresholds(sample_cand)
    rmin = bat_threshold(sample_cand)
    print("CZL thresholds ->", rho_lo, rho_hi, rho_mid)
    print("BAT risk_min ->", rmin)
else:
    print("Skip thresholds: no candidates file.")


In [ ]:

if sample_cand:
    czl_out = run_czl_orb(sample_cand)
    bat_out = run_bat_orb(sample_cand)
    print("CZL output:", czl_out)
    print("BAT output:", bat_out)
else:
    print("Skip CZL/BAT: no candidates file.")


In [ ]:

if sample_edge and sample_timeline:
    graph_problem = load_graph_problem(sample_edge, sample_timeline)
    print("Graph graph_id:", graph_problem.graph_id)
    print("Edges df shape:", graph_problem.edges_df.shape)
    print("Timeline shape:", graph_problem.timeline_df.shape)
else:
    print("No edge/timeline file found. Update paths or generate offline problems.")


In [ ]:

if sample_edge and sample_timeline:
    itm_res = run_itm_online(sample_edge, sample_timeline, threshold=0.0)
    print("ITM result:", itm_res)
else:
    print("Skip ITM: no edge/timeline file.")
